In [ ]:
import json

# Чтение исходного файла
with open("extract_flibusta_dialogues.1.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

# Сбор диалогов в пары
dialogues = []
current_input = None

for line in lines[:404741]:
    if line.startswith("- "):
        text = line[2:].strip()  # Удаляем дефис и пробел
        if current_input is None:
            current_input = text
        else:
            dialogues.append({"input": current_input, "output": text})
            current_input = None

# Сохранение в JSON
with open("processed_dialogues.json", "w", encoding="utf-8") as f:
    json.dump(dialogues, f, indent=2, ensure_ascii=False)

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes gradio
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    Trainer,
    DataCollatorForLanguageModeling  # Добавляем data collator
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [ ]:
# Загрузка модели
model_name = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # Оптимизация для GPU
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset


# Загрузка данных
with open("processed_dialogues.json", "r", encoding="utf-8") as f:
    dialogues = json.load(f)

dataset = Dataset.from_list(dialogues)
dataset = dataset.map(lambda x: {"text": f"<s>user: {x['input']}\nbot: {x['output']}</s>"})

with open("processed_dialogues.json", "r", encoding="utf-8") as f:
    dialogues = json.load(f)

# Формат: "user: [вопрос]\nbot: [ответ]"
dataset = Dataset.from_list(dialogues)
dataset = dataset.map(lambda x: {
    "text": f"user: {x['input']}\nbot: {x['output']}<|endoftext|>"  # Спецтокен для GPT
})

def tokenize(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=64,
        return_tensors="pt",
        padding="max_length"  # GPT-2 требует фиксированную длину
    )

dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/124429 [00:00<?, ? examples/s]

Map:   0%|          | 0/124429 [00:00<?, ? examples/s]

Map:   0%|          | 0/124429 [00:00<?, ? examples/s]

In [ ]:
torch.cuda.empty_cache()

# Настройка переменных окружения
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"], # Слои внимания GPT-2
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=3e-4,
    fp16=True,
    optim="adamw_torch",
    save_strategy="no",
    report_to="none",
    logging_steps=1000
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)
trainer.train()
model.save_pretrained("gpt2-memory-bot")



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1000,1.928700
2000,1.763400
3000,1.751500


'\n# Настройка LoRA\npeft_config = LoraConfig(\n    r=8,\n    lora_alpha=32,\n    target_modules=["q_proj", "v_proj"],\n    lora_dropout=0.05,\n    bias="none",\n    task_type="CAUSAL_LM"\n)\nmodel = get_peft_model(model, peft_config)\n\ndef preprocess(examples):\n    tokenized = tokenizer(\n        examples["text"],\n        truncation=True,\n        max_length=128,\n        padding="max_length",\n        add_special_tokens=True\n    )\n    # Явно добавляем labels = input_ids для вычисления loss\n    tokenized["labels"] = tokenized["input_ids"].copy()\n    return tokenized\n\ntokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])\n\n# Обучение\ntraining_args = TrainingArguments(\n    output_dir="output",\n    per_device_train_batch_size=2,\n    gradient_accumulation_steps=4,\n    num_train_epochs=5,\n    learning_rate=3e-5,\n    fp16=True,\n    logging_steps=10,\n)\n\ndef pad_to_max_length(sequences, max_length):\n    return [seq + [0] * (max_length - len(s

In [ ]:
# Демо
model = AutoModelForCausalLM.from_pretrained("gpt2-memory-bot")
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_response(input_text):
    prompt = f"<s>user: {input_text}\nbot:"
    response = generator(
        prompt,
        max_length=200,
        temperature=0.7,
        top_p=0.85,
        repetition_penalty=1.1
    )[0]["generated_text"]
    return response.split("bot:")[-1].strip().replace("</s>", "")

gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Напишите сообщение..."),
    outputs="text",
    title="🕊️ Memory Bot",
    description="Задайте вопрос, чтобы получить ответ в стиле вашего диалога."
).launch(share=True)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6c311132b0f8b4c07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
